---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [1]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

# Summer  Gold  Silver  Bronze  Total  # Winter  Gold.1  \
Afghanistan        13     0       0       2      2         0       0   
Algeria            12     5       2       8     15         3       0   
Argentina          23    18      24      28     70        18       0   
Armenia             5     1       2       9     12         6       0   
Australasia         2     3       4       5     12         0       0   

             Silver.1  Bronze.1  Total.1  # Games  Gold.2  Silver.2  Bronze.2  \
Afghanistan         0         0        0       13       0         0         2   
Algeria             0         0        0       15       5         2         8   
Argentina           0         0        0       41      18        24        28   
Armenia             0         0        0       11       1         2         9   
Australasia         0         0        0        2       3         4         5   

             Combined total   ID  
Afghanistan               2  AFG  
Algeria                  15  ALG  
Argentina                70  ARG  
Armenia                  12  ARM  
Australasia              12  ANZ

### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [2]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [3]:
answer=df.index[df['Gold']==max(df['Gold'])]
type(answer[0])

str

In [2]:
def answer_one():    
    
    return  df[df['Gold']==max(df['Gold'])].index[0]

answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [5]:
def answer_two():
    
    return df.index[df['Gold']-df['Gold.1']==max(df['Gold']-df['Gold.1'])][0]

answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [6]:
df1=df[(df['Gold']>0)&(df['Gold.1']>0)]
df1['prop']=(df1['Gold']-df1['Gold.1'])/(df1['Gold']+df1['Gold.1'])
df1.head()
df1[df1.index=='Bulgaria']

# Summer  Gold  Silver  Bronze  Total  # Winter  Gold.1  Silver.1  \
Bulgaria        19    51      85      78    214        19       1         2   

          Bronze.1  Total.1  # Games  Gold.2  Silver.2  Bronze.2  \
Bulgaria         3        6       38      52        87        81   

          Combined total   ID      prop  
Bulgaria             220  BUL  0.961538

In [7]:
def answer_three():
    
    df1=df[(df['Gold']>0)&(df['Gold.1']>0)]
    df1['prop']=(df1['Gold']-df1['Gold.1'])/(df1['Gold.2'])
    return df1.index[df1['prop']==max(df1['prop'])][0]

answer_three()

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [8]:
def answer_four():
    Points=df['Gold.2']*3+df['Silver.2']*2+df['Bronze.2']
    return Points


## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [3]:
census_df = pd.read_csv('census.csv')
census_df.head()

SUMLEV  REGION  DIVISION  STATE  COUNTY   STNAME         CTYNAME  \
0      40       3         6      1       0  Alabama         Alabama   
1      50       3         6      1       1  Alabama  Autauga County   
2      50       3         6      1       3  Alabama  Baldwin County   
3      50       3         6      1       5  Alabama  Barbour County   
4      50       3         6      1       7  Alabama     Bibb County   

   CENSUS2010POP  ESTIMATESBASE2010  POPESTIMATE2010     ...       \
0        4779736            4780127          4785161     ...        
1          54571              54571            54660     ...        
2         182265             182265           183193     ...        
3          27457              27457            27341     ...        
4          22915              22919            22861     ...        

   RDOMESTICMIG2011  RDOMESTICMIG2012  RDOMESTICMIG2013  RDOMESTICMIG2014  \
0          0.002295         -0.193196          0.381066          0.582002   
1          7.242091         -2.915927         -3.012349          2.265971   
2         14.832960         17.647293         21.845705         19.243287   
3         -4.728132         -2.500690         -7.056824         -3.904217   
4         -5.527043         -5.068871         -6.201001         -0.177537   

   RDOMESTICMIG2015  RNETMIG2011  RNETMIG2012  RNETMIG2013  RNETMIG2014  \
0         -0.467369     1.030015     0.826644     1.383282     1.724718   
1         -2.530799     7.606016    -2.626146    -2.722002     2.592270   
2         17.197872    15.844176    18.559627    22.727626    20.317142   
3        -10.543299    -4.874741    -2.758113    -7.167664    -3.978583   
4          0.177258    -5.088389    -4.363636    -5.403729     0.754533   

   RNETMIG2015  
0     0.712594  
1    -2.187333  
2    18.293499  
3   -10.543299  
4     1.107861  

[5 rows x 100 columns]

In [9]:
counties_df = census_df[census_df['SUMLEV'] == 50]
x=counties_df.groupby('STNAME').count()['CTYNAME']
x.idxmax()

'Texas'

In [10]:
def answer_five():
    
    df=census_df[census_df['SUMLEV'] == 50]

    df1 = pd.DataFrame()
    df1['State']=df['STNAME'].unique()


    df1['# Counties']=0
    name='Alabama'

    for name in df1['State']:
        df1['# Counties'][df1['State']==name]=len(df[df['STNAME']==name])

    df1=df1.set_index('State')
    
    return df1.index[df1['# Counties']==max(df1['# Counties'])][0]

answer_five()

'Texas'

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [12]:

counties_df = census_df[census_df['SUMLEV'] == 50]

top_counties_df = counties_df.sort_values(by = 'CENSUS2010POP', ascending = False).groupby('STNAME').head(3)
#     print(top_counties_df)
    
#     top_counties_df = counties_df.sort_values(by=['CENSUS2010POP'],ascending=False).groupby('STNAME').head(3)
#     print(top_counties_df)
ans = top_counties_df.groupby('STNAME').sum().sort_values(by='CENSUS2010POP' , ascending = False).head(3).index.tolist()

top_counties_df

SUMLEV  REGION  DIVISION  STATE  COUNTY         STNAME  \
209       50       4         9      6      37     California   
624       50       2         3     17      31       Illinois   
2667      50       3         7     48     201          Texas   
106       50       4         8      4      13        Arizona   
227       50       4         9      6      73     California   
220       50       4         9      6      59     California   
1884      50       1         2     36      47       New York   
372       50       3         5     12      86        Florida   
2623      50       3         7     48     113          Texas   
1901      50       1         2     36      81       New York   
1777      50       4         8     32       3         Nevada   
3017      50       4         9     53      33     Washington   
1335      50       2         3     26     163       Michigan   
2786      50       3         7     48     439          Texas   
335       50       3         5     12      11        Florida   
1891      50       1         2     36      61       New York   
2333      50       1         2     42     101   Pennsylvania   
1247      50       1         1     25      17  Massachusetts   
379       50       3         5     12      99        Florida   
2096      50       2         3     39      35           Ohio   
2284      50       1         2     42       3   Pennsylvania   
1316      50       2         3     26     125       Michigan   
2103      50       2         3     39      49           Ohio   
1364      50       2         4     27      53      Minnesota   
2895      50       3         5     51      59       Virginia   
2839      50       4         8     49      35           Utah   
1604      50       2         4     29     189       Missouri   
109       50       4         8      4      19        Arizona   
1228      50       3         5     24      31       Maryland   
559       50       4         9     15       3         Hawaii   
...      ...     ...       ...    ...     ...            ...   
886       50       2         4     19     163           Iowa   
324       50       3         5     10       1       Delaware   
1442      50       3         6     28      33    Mississippi   
1069      50       3         6     21     117       Kentucky   
1758      50       2         4     31     153       Nebraska   
2855      50       1         1     50       7        Vermont   
562       50       4         9     15       9         Hawaii   
1206      50       1         1     23      19          Maine   
2033      50       2         4     38      17   North Dakota   
1680      50       4         8     30     111        Montana   
1800      50       1         1     33      13  New Hampshire   
1853      50       4         8     35      49     New Mexico   
591       50       4         8     16      55          Idaho   
2355      50       1         1     44       9   Rhode Island   
1656      50       4         8     30      63        Montana   
3042      50       3         5     54       3  West Virginia   
2455      50       2         4     46     103   South Dakota   
76        50       4         9      2      90         Alaska   
3046      50       3         5     54      11  West Virginia   
3180      50       4         8     56      21        Wyoming   
1639      50       4         8     30      29        Montana   
85        50       4         9      2     170         Alaska   
2032      50       2         4     38      15   North Dakota   
3182      50       4         8     56      25        Wyoming   
2042      50       2         4     38      35   North Dakota   
2862      50       1         1     50      21        Vermont   
2863      50       1         1     50      23        Vermont   
1792      50       4         8     32     510         Nevada   
3172      50       4         8     56       5        Wyoming   
2444      50       2         4     46      83   South Dakota   

                           CTYNAME  C

In [73]:
def answer_six():
    df=census_df[census_df['SUMLEV'] == 50]

    columns_to_keep = ['STNAME',
                       'CTYNAME',
                       'CENSUS2010POP'
                       ]

    df = df[columns_to_keep]

    df1 = pd.DataFrame()
    df1['State']=df['STNAME'].unique()
    df1['Population']=0
    #df1.head()

    df_test=df[df['STNAME']=='Alabama'].sort_values(by=['CENSUS2010POP'],ascending=False)


    df_test.iloc[:3,df_test.columns.get_loc('CENSUS2010POP')].sum()

    for name in df1['State']:
            df_temp=df[df['STNAME']==name].sort_values(by=['CENSUS2010POP'],ascending=False)
            df1['Population'][df1['State']==name]=df_temp.iloc[:3,df_test.columns.get_loc('CENSUS2010POP')].sum()
        

    str(df1.sort_values(by=['Population'],ascending=False)[:3])

    
    return ['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [12]:
def answer_seven():
    
    df=census_df[census_df['SUMLEV'] == 50]

    columns_to_keep = ['CTYNAME',
                   'POPESTIMATE2015',
                   'POPESTIMATE2014',
                   'POPESTIMATE2013',
                   'POPESTIMATE2012',
                   'POPESTIMATE2011',
                   'POPESTIMATE2010'
                   ]
    df = df[columns_to_keep]

    df = df.set_index(['CTYNAME'])
    df['max']=df.max(axis=1)
    df['min']=df.min(axis=1)
    df['diff']=df['max']-df['min']
    #df['max']=max(df['POPESTIMATE2015'],df['POPESTIMATE2014'],df['POPESTIMATE2013'],df['POPESTIMATE2012'],df['POPESTIMATE2011'],df['POPESTIMATE2010'])

    return df.index[df['diff']==df['diff'].max()][0]

answer_seven()



'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [11]:
def answer_eight():
    df=census_df[census_df['SUMLEV'] == 50]

    columns_to_keep = ['STNAME',
                       'CTYNAME',
                       'REGION',
                       'POPESTIMATE2015',
                       'POPESTIMATE2014'
                   ]
    df = df[columns_to_keep]

    df1=df[((df['REGION']==1) | (df['REGION']==2)) 
           & (df['POPESTIMATE2015']>df['POPESTIMATE2014']) 
           & df['CTYNAME'].str.match('Washington')]

    df1

    columns_to_keep1 = ['STNAME',
                       'CTYNAME'
                   ]
   
    return df1[columns_to_keep1]